In [1]:
import pennylane as qml
from pennylane import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import cv2
import random
import os
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import random
from PIL import Image

In [2]:
def get_category_dict(dataset_path):
    categories = sorted([
        folder for folder in os.listdir(dataset_path)
        if os.path.isdir(os.path.join(dataset_path, folder))
    ])
    return {category: idx for idx, category in enumerate(categories)}


In [3]:
dataset = "Food_Classification"
categories = get_category_dict(dataset)
num_images = 500
image_size = (256, 256) 

In [4]:
def nearest_power_of_2(n):
    return 2 ** int(np.ceil(np.log2(n)))

In [5]:
for category in categories:
    folder_path = os.path.join(dataset, category)
    images = os.listdir(folder_path)[:num_images] 

quantum_encoded_images = []

In [6]:
for category in categories:
    folder_path = os.path.join(dataset, category)
    image_paths = [os.path.join(folder_path, img) for img in images]
    images = os.listdir(folder_path)[:num_images] 

    for img_file in tqdm(images, desc=f"Encoding {category} images"):
        try:
            
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, image_size)  
            image = image.flatten() / 256.0 

            target_size = nearest_power_of_2(len(image)) 
            num_qubits = int(np.log2(target_size))  


            dev = qml.device("default.qubit", wires=num_qubits)

            @qml.qnode(dev)
            def quantum_image_encoding(image):
                qml.AmplitudeEmbedding(features=image, wires=range(num_qubits), normalize=True, pad_with=0.0)
                return qml.state()

            padded_image = np.pad(image, (0, target_size - len(image)), 'constant')

            quantum_state = quantum_image_encoding(padded_image)
            quantum_encoded_images.append((category, img_file, quantum_state))
        
        except Exception as e:
            print(f"Error processing {img_file}: {e}")


Encoding samosa images: 100%|██████████| 262/262 [00:04<00:00, 59.42it/s]


In [7]:
with open("quantum_encoded_pet_images.pkl", "wb") as f:
    pickle.dump(quantum_encoded_images, f)

print("Encoding Sucessfull!")

Encoding Sucessfull!


In [8]:
with open("quantum_encoded_pet_images.pkl", "rb") as f:
    loaded_data = pickle.load(f)

In [9]:
random_entries = random.sample(loaded_data, 10)

for i, (category, filename, quantum_state) in enumerate(random_entries):
    print(f"\n Entry {i+1}")
    print(f"Category : {category}")
    print(f"Filename : {filename}")
    print(f" Quantum State (Size: {len(quantum_state)}):")
    print(quantum_state[:10])


 Entry 1
Category : chai
Filename : 011.jpg
 Quantum State (Size: 65536):
[6.63196945e-05+0.j 6.63196945e-05+0.j 6.63196945e-05+0.j
 6.63196945e-05+0.j 6.63196945e-05+0.j 6.63196945e-05+0.j
 6.63196945e-05+0.j 6.63196945e-05+0.j 6.63196945e-05+0.j
 6.63196945e-05+0.j]

 Entry 2
Category : kaathi_rolls
Filename : 139.jpg
 Quantum State (Size: 65536):
[0.00471972+0.j 0.00475825+0.j 0.00473898+0.j 0.00475825+0.j
 0.00479678+0.j 0.00479678+0.j 0.00483531+0.j 0.00481604+0.j
 0.00483531+0.j 0.00487383+0.j]

 Entry 3
Category : masala_dosa
Filename : 210.jpg
 Quantum State (Size: 65536):
[0.00280376+0.j 0.00259497+0.j 0.00259497+0.j 0.00256514+0.j
 0.002416  +0.j 0.002416  +0.j 0.00235635+0.j 0.0022967 +0.j
 0.00223704+0.j 0.00220721+0.j]

 Entry 4
Category : kulfi
Filename : 185.jpg
 Quantum State (Size: 65536):
[0.00364213+0.j 0.00392908+0.j 0.00403945+0.j 0.00410567+0.j
 0.00417189+0.j 0.00461336+0.j 0.0050769 +0.j 0.00527557+0.j
 0.00450299+0.j 0.00240601+0.j]

 Entry 5
Category : dhokla